In [7]:
import pandas as pd
from datetime import datetime

# Load the data
file_path = 'new_Search Result in Jakarta _ Travelio.com.csv'
data = pd.read_csv(file_path)

# Cleaning the dataset

# Strip whitespace and newline characters from all string fields
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove columns Price, Info, and Image
data = data.drop(columns=['Price', 'Info', 'Image'])

# Clean the `Description` column to extract the city (first part before a comma)
data['Description'] = data['Description'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else x)

# Clean the `Tag/Capacity` column by removing non-numeric characters
data['Tag'] = data['Tag'].str.extract('(\d+)').astype(float)

# Extract the unit number from the 'Unit' column
data['Unit'] = data['Unit'].str.extract('(\d+)').astype(float)

# Standardize and clean `dflex` column (removing redundant parts)
data['dflex'] = data['dflex'].str.replace('IDR\n', '').str.replace('\n / bulan', '').str.replace(',', '')
data['dflex'] = pd.to_numeric(data['dflex'], errors='coerce')

data['Tag'] = data['Tag'].fillna(0)
data['Type'] = data['Type'].fillna('Unfurnished')
# data['propertysmallicon4'] = data['propertysmallicon4'].str.replace('New', '')
data['propertysmallicon4'] = data['propertysmallicon4'].fillna(0)
data['Label'] = data['Label'].fillna('tidak ada promo')


# Add columns for month, day, year, and quarter based on the current date
current_date = datetime.now()
data['date'] = current_date.strftime('%Y-%m-%d')
data['month'] = current_date.month
data['day'] = current_date.day
data['year'] = current_date.year
data['quarter'] = (current_date.month - 1) // 3 + 1  # Calculate quarter

# Add a new column 'rental_period' with the literal word "bulan"
data['rental_period'] = 'monthly'

# Renaming columns for clarity and converting to snake_case
data = data.rename(columns={
    'Title': 'property_title',
    'Unit': 'availability_number',
    'Label': 'promotion_label',
    'Description': 'city',
    'propertysmallicon': 'property_type',
    'propertysmallicon1': 'bedrooms',
    'propertysmallicon2': 'bathrooms',
    'propertysmallicon3': 'apart_size',
    'propertysmallicon4': 'rating',
    'Tag': 'capacity',
    'Type': 'furnish_type',
    'dflex': 'price',
})


# Handle bathrooms containing "m2" or "sqm": Append to apartsize and set bathrooms to 1
mask_bathrooms = data['bathrooms'].str.contains(r'\b(m2|sqm)\b', na=False)
data['apart_size'] = data['apart_size'].fillna('') + data['bathrooms'].where(mask_bathrooms, '').astype(str)
data['bathrooms'] = data['bathrooms'].mask(mask_bathrooms, 1)

# property_type ke bedroom
mask_numeric = data['property_type'].str.contains(r'^[1-5]$', na=False)
data.loc[mask_numeric, 'bedrooms'] = data.loc[mask_numeric, 'property_type']
data.loc[mask_numeric, 'property_type'] = 'Apartment'

data['property_type'] = data['property_type'].replace('Studio', 'Apartment')
data['apart_size'] = data['apart_size'].str.strip().str.replace(r'\s*m2$', ' sqm', regex=True)

# export data
cleaned_file_path = 'cleaned_travelio_data_city_split_2.csv'
data.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data with city split saved to {cleaned_file_path}")

Cleaned data with city split saved to cleaned_travelio_data_city_split_2.csv


<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:23: SyntaxWarning: invalid escape sequence '\d'
C:\Users\adria\AppData\Local\Temp\ipykernel_5656\3439360176.py:20: SyntaxWarning: invalid escape sequence '\d'
  data['Tag'] = data['Tag'].str.extract('(\d+)').astype(float)
C:\Users\adria\AppData\Local\Temp\ipykernel_5656\3439360176.py:23: SyntaxWarning: invalid escape sequence '\d'
  data['Unit'] = data['Unit'].str.extract('(\d+)').astype(float)
C:\Users\adria\AppData\Local\Temp\ipykernel_5656\3439360176.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
C:\Users\adria\AppData\Local\Temp\ipykernel_5656\3439360176.py:65: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_bathrooms = data[

In [13]:
df = pd.read_csv('cleaned_travelio_data_city_split_2.csv')
df.head()

,property_title,availability_number,promotion_label,property_type,bedrooms,bathrooms,furnish_type,apart_size,rating,city,capacity,price,currency,date,month,day,year,quarter,rental_period
0,Classic 2BR City Home Gading Riverview (MOI) A...,1.0,Flash Sale,Apartment,2,1,Full Furnished,45 sqm,5.0,Kelapa Gading,3.0,5890000,IDR,2025-01-16,1,16,2025,1,monthly
1,2BR Unfurnished with AC at 9th Floor Green Par...,1.0,Limited Time,Apartment,2,1,Unfurnished,36 sqm,0,Daan Mogot,0.0,2935500,IDR,2025-01-16,1,16,2025,1,monthly
2,Stunning 2BR with Private Lift at Masterpiece ...,1.0,Up To 1JT,Apartment,2,2,Full Furnished,83 sqm,4.9,Epicentrum,3.0,12750000,IDR,2025-01-16,1,16,2025,1,monthly
3,Good Deal and Restful Studio Bassura City Apar...,1.0,tidak ada promo,Apartment,Studio,1,Full Furnished,18 sqm,5.0,Cipinang,2.0,3860000,IDR,2025-01-16,1,16,2025,1,monthly
4,Stunning and Spacious 1BR at Casablanca Apartm...,1.0,tidak ada promo,Apartment,1,1,Full Furnished,77 sqm,4.6,Casablanca,2.0,13000000,IDR,2025-01-16,1,16,2025,1,monthly


In [56]:
df.isna().sum()

property_title         0
availability_number    0
promotion_label        0
property_type          0
bedrooms               0
bathrooms              0
furnish_type           0
apart_size             0
rating                 0
city                   0
capacity               0
price                  0
currency               0
bulan                  0
tanggal                0
tahun                  0
quarter                0
rental_period          0
dtype: int64

In [57]:
df[df.duplicated()]

,property_title,availability_number,promotion_label,property_type,bedrooms,bathrooms,furnish_type,apart_size,rating,city,capacity,price,currency,bulan,tanggal,tahun,quarter,rental_period


In [70]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   property_title       1122 non-null   object 
 1   availability_number  1122 non-null   float64
 2   promotion_label      1122 non-null   object 
 3   property_type        1122 non-null   object 
 4   bedrooms             1122 non-null   object 
 5   bathrooms            1122 non-null   int64  
 6   furnish_type         1122 non-null   object 
 7   apart_size           1122 non-null   object 
 8   rating               1122 non-null   object 
 9   city                 1122 non-null   object 
 10  capacity             1122 non-null   float64
 11  price                1122 non-null   int64  
 12  currency             1122 non-null   object 
 13  date                 1122 non-null   object 
 14  month                1122 non-null   int64  
 15  day                  1122 non-null   i

In [51]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.8 MB/s eta 0:00:00


In [1]:
import psycopg2
import pandas as pd

In [4]:
# Konfigurasi koneksi PostgreSQL
conn_params = {
    "host": "ep-dawn-sun-a52rqf4q.us-east-2.aws.neon.tech",  # Ganti dengan host PostgreSQL Anda
    "database": "travelio",  # Ganti dengan nama database Anda
    "user": "neondb_owner",  # Ganti dengan username PostgreSQL Anda
    "password": "MNyJ6fn2wIoA",  # Ganti dengan password PostgreSQL Anda
    "port": 5432  # Default port PostgreSQL
}


# Koneksi ke PostgreSQL
try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    print("Koneksi berhasil ke PostgreSQL!")
except Exception as e:
    print(f"Error saat koneksi: {e}")
    exit()

Koneksi berhasil ke PostgreSQL!


In [74]:
def create_tables():
    queries = [
        """
        CREATE TABLE IF NOT EXISTS dim_property (
            property_id SERIAL PRIMARY KEY,
            property_title VARCHAR(255),
            city VARCHAR(100),
            property_type VARCHAR(50),
            bedrooms VARCHAR(50),
            bathrooms INT,
            rating VARCHAR(50),
            furnish_type VARCHAR(50),
            apart_size VARCHAR(50),
            promotion_label VARCHAR(255)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS dim_time (
            date_id SERIAL PRIMARY KEY,
            date DATE,
            year INT,
            month INT,
            day INT,
            quarter INT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS dim_capacity (
            capacity_id SERIAL PRIMARY KEY,
            capacity INT,
            rental_period VARCHAR(50)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS fact_property_rentals (
            fact_id SERIAL PRIMARY KEY,
            price FLOAT,
            availability_number INT
        );
        """
    ]
    try:
        for query in queries:
            cursor.execute(query)
        conn.commit()
        print("Tabel berhasil dibuat!")
    except Exception as e:
        print(f"Error saat membuat tabel: {e}")
        conn.rollback()


create_tables()

Tabel berhasil dibuat!


In [5]:
def create_tables():
    queries = [
        """
        CREATE TABLE IF NOT EXISTS all_data (
            property_title VARCHAR(255),
            city VARCHAR(100),
            property_type VARCHAR(50),
            bedrooms VARCHAR(50),
            bathrooms INT,
            rating VARCHAR(50),
            furnish_type VARCHAR(50),
            apart_size VARCHAR(50),
            promotion_label VARCHAR(255),
            date DATE,
            year INT,
            month INT,
            day INT,
            quarter INT,
            capacity INT,
            rental_period VARCHAR(50),
            price FLOAT,
            availability_number INT 
        );
        """
    ]
    try:
        for query in queries:
            cursor.execute(query)
        conn.commit()
        print("Tabel berhasil dibuat!")
    except Exception as e:
        print(f"Error saat membuat tabel: {e}")
        conn.rollback()

create_tables()

Tabel berhasil dibuat!


In [8]:
# Memaasukkan data ke tabel PostgreSQL
def insert_data_to_postgres(df, table_name):
    # Konversi DataFrame ke list of tuples dengan tipe data Python bawaan
    records_list = df.astype(object).where(pd.notnull(df), None).values.tolist()

    # Generate placeholder untuk query
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    
    # Query SQL untuk insert
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    
    try:
        cursor.executemany(query, records_list)
        conn.commit()
        print(f"Data berhasil dimasukkan ke tabel {table_name}.")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")
        conn.rollback()


# Masukkan data ke tabel sesuai skema
# Dimensi waktu
# dim_time = data[['date', 'year', 'month', 'day', 'quarter']].drop_duplicates()
# insert_data_to_postgres(dim_time, "dim_time")

# # Dimensi properti
# dim_property = data[['property_title', 'city', 'property_type', 'bedrooms',
#                      'bathrooms', 'rating', 'furnish_type', 'apart_size', 'promotion_label']].drop_duplicates()
# insert_data_to_postgres(dim_property, "dim_property")

# # Dimensi kapasitas
# dim_capacity = data[['capacity', 'rental_period']].drop_duplicates()
# insert_data_to_postgres(dim_capacity, "dim_capacity")

all_data = data[[
    'property_title', 
    'city', 
    'property_type', 
    'bedrooms', 
    'bathrooms', 
    'rating', 
    'furnish_type', 
    'apart_size', 
    'promotion_label', 
    'date', 'year', 
    'month', 
    'day', 
    'quarter', 
    'capacity', 
    'rental_period', 
    'price', 
    'availability_number']]

insert_data_to_postgres(all_data, "all_data")

# # Tabel fakta
# fact_property_rentals = data[['price', 'availability_number']]
# insert_data_to_postgres(fact_property_rentals, "fact_property_rentals")


# Tutup koneksi
cursor.close()
conn.close()
print("Proses selesai, koneksi ditutup!")

Data berhasil dimasukkan ke tabel all_data.
Proses selesai, koneksi ditutup!
